In [927]:
import numpy as np
import pandas as pd

In [928]:
class Hmm:
    def __init__(self,file,k=3):
        self.words = None #set of unique words
        self.tag_word_count = None # dict((tag,word),count)
        self.transmissions = None # dict((tag_u,tag_v),count)
        self.count = None # dict(tag,count)
        self.read_file(file,k)
        self.tags = set(self.count.keys())
        
        self.word_ls = tuple(self.words)#tuple cause immutable
        self._tag_ls = tuple(self.tags)
        #move #START# to first row and #END# to last row
        ls = list(self._tag_ls)
        ls.remove('#START#')
        ls.remove('#END#')
        ls.insert(0,'#START#')
        ls.append('#END#')
        self.tag_ls = tuple(ls)
        
        self.make_matrix()
    
    def make_matrix(self):
        tag_length = len(self.tag_ls)
        transition_matrix = np.zeros((tag_length,tag_length))
        
        for i in range(tag_length):
            for j in range(tag_length):
                tag_u = self.tag_ls[i]
                tag_v = self.tag_ls[j]
                transition_matrix[i][j] = self.transmissions[(tag_u,tag_v)]/self.count[tag_u]
        self.transition_matrix = pd.DataFrame(transition_matrix,index=self.tag_ls,columns=self.tag_ls)
        
        word_length = len(self.word_ls)
        em_matrix = np.zeros((tag_length,word_length))
        for i in range(tag_length):
            for j in range(word_length):
                tag = self.tag_ls[i]
                word = self.word_ls[j]
                em_matrix[i][j] = self.tag_word_count[(tag,word)]/self.count[tag]
        self.em_matrix = pd.DataFrame(em_matrix,index=self.tag_ls,columns=self.word_ls)
        pass
    
    def read_file(self,file,k):
        from collections import defaultdict
        seq = ['#START#']
        f = open(file,'r',encoding='UTF-8')
        tag_word_ls = []
        word_count = defaultdict(int)
        for line in f:
            split = line.split(' ')
            if len(split)<2:
                #this is a line break
                seq.append('#END#')
                seq.append('#START#')
                continue
            word,tag = split
            word = word.strip()
            tag = tag.strip()
            tag_word_ls.append([tag,word])
            word_count[word]+=1
            seq.append(tag)
        f.close()
        
        #Emissions
        for i in range(len(tag_word_ls)):
            tag,word = tag_word_ls[i]
            if word_count[word]<k:
                tag_word_ls[i] = [tag,'#UNK#']
        tag_word_count = defaultdict(int)
        
        words = []
        for tag,word in tag_word_ls:
            tag_word_count[tag,word]+=1
            words.append(word)
        self.words = set(words)
        self.tag_word_count= tag_word_count
        
        #Transistions
        del seq[-1] #delete last item from the list
         #print(seq)
        trans_dict = defaultdict(int)
        count_u = defaultdict(int)
        for i in range(len(seq)-1):
            tag_u = seq[i]
            count_u[tag_u] += 1 # need to count #END# too
            if tag_u == "#END#":
                continue
            #if u is not #END# we count the transmission 
            tag_v = seq[i+1]
            if (tag_u =="#START#" and tag_v == "#END#"):
                #check for empty blank lines at the end and dont count them
                print('these are blank lines')
                count_u["#START#"] -= 1 #remove additional start
                break
            trans_dict[(tag_u,tag_v)] += 1
        self.transmissions = trans_dict
        self.count = count_u

In [929]:
EN = Hmm('./EN/train')

In [930]:
EN.em_matrix.loc['O','.']

0.3120392091152815

In [931]:
EN.transition_matrix.loc['#START#','B-NP']

0.6480490669450607

In [966]:
def log(m):
    m = np.clip(m, 1e-32, None)
    x = np.log(m)
    return x

In [1020]:
def vertebi(word_arr,Hmm):
    """
    Followed pseudocode here
    https://en.wikipedia.org/wiki/Viterbi_algorithm#Pseudocode
    """
    S = Hmm.tag_ls[1:-1] #set of all possible tags remove #START# and #STOP#
    
    A = Hmm.transition_matrix.values[1:-1,1:-1] # A(tag_u_vector,tag_v)
    B = Hmm.em_matrix[1:-1] # B(tag_u->word)
    
    T = len(S) # Total number unique tags
    N = len(word_arr) # Length of sentence make sure no #START# and #STOP#
    
    T1 = np.zeros((T,N)) #probability table of most possible path to node i.e. store scores of each node
    T2 = np.zeros((T,N)) # Table of paths where the ith row stores highest scoring paths to T1[i,j]
    
    #Handle first word and base case at the same time
    word = word_arr[0]
    if word not in Hmm.words:
        word = '#UNK#'
    T1[:,0] = 1*log(Hmm.transition_matrix.loc['#START#'][1:-1].values*B[word].values)
    #Note A is vector operation
    # Fill up each column by using previous column
    # j is position of word
    
    for j in range(1,N):
        # i is position of tag
        #ignore #START# and #END# tag when looping
        for i in range(T):
            tag = S[i]
            #note A(S,tag_u gives a vector)
            word = word_arr[j]
            if word not in Hmm.words:
                word = '#UNK#'
            T1[i][j] = np.max(T1[:,j-1]+log(A[:,i]*B.loc[tag,word]))
            T2[i][j] = np.argmax(T1[:,j-1]+log(A[:,i]))
    #handle last word to #END#
    #no emission of #END# 
#     print(T1)
#     print(T2)
#     print(T1[:,N-1]*A(S,'#END#'))
    
    best_row = np.argmax(T1[:,N-1]+log(Hmm.transition_matrix['#END#'].values[1:-1]))
    ans=[]
    curr_index = best_row
    ans.append(S[curr_index])
    for j in range(N-1,0,-1):
        prev_index = T2[int(curr_index)][j]
        ans.append(S[int(prev_index)])
        curr_index = prev_index
#         print(S[j])
    ans = ans[::-1]
#     print(T1)
    return ans

In [1021]:
word = "Trump is the best president in the world".split(' ')
len(word)

8

In [1022]:
vertebi(word,EN)

['B-ADVP', 'B-VP', 'B-NP', 'I-NP', 'I-NP', 'B-PP', 'B-NP', 'I-NP']

In [1014]:
LANG = ['AL','CN','EN','SG']
eval_params = lambda lang: {'devin':f'./{lang}/dev.in','devout':f'./{lang}/dev.p3.out','ground_truth':f'./{lang}/dev.out','trainfile':f'./{lang}/train'}

In [1015]:
import os
def pred_out(devin,devout,ground_truth,trainfile):
    H = Hmm(trainfile)
    file_object = open(devin, "r",encoding='UTF-8',)
    ls=[[]]
    index=0
    test=[]
    for line in file_object:
        test.append(line.strip())
        if (line.strip()==""):
            ls.append([])
            index+=1
        else:
            ls[index].append(line.strip())
    ls.pop(-1)
    df = pd.DataFrame(test, columns = ['Word'])
    
    from tqdm.notebook import tqdm
    predict=[]
    for i in tqdm(ls):
        for j in vertebi(i,H):
            predict.append(j)
        predict.append("")
    df['Tag'] = predict
    
    df.to_csv(devout, sep=" ", index=False, header=False)
    
    if os.name == 'nt':#if it is on windows
        !python ./EvalScript/evalResult.py {ground_truth} {devout}
    else:
        !python3 ./EvalScript/evalResult.py {ground_truth} {devout}

In [1016]:
for lang in LANG:
    pred_out(**eval_params(lang))



#Entity in gold data: 8408
#Entity in prediction: 8498

#Correct Entity : 6740
Entity  precision: 0.7931
Entity  recall: 0.8016
Entity  F: 0.7974

#Correct Sentiment : 6087
Sentiment  precision: 0.7163
Sentiment  recall: 0.7240
Sentiment  F: 0.7201




#Entity in gold data: 1478
#Entity in prediction: 712

#Correct Entity : 307
Entity  precision: 0.4312
Entity  recall: 0.2077
Entity  F: 0.2804

#Correct Sentiment : 210
Sentiment  precision: 0.2949
Sentiment  recall: 0.1421
Sentiment  F: 0.1918




#Entity in gold data: 13179
#Entity in prediction: 13060

#Correct Entity : 11079
Entity  precision: 0.8483
Entity  recall: 0.8407
Entity  F: 0.8445

#Correct Sentiment : 10651
Sentiment  precision: 0.8155
Sentiment  recall: 0.8082
Sentiment  F: 0.8118




#Entity in gold data: 4537
#Entity in prediction: 3008

#Correct Entity : 1665
Entity  precision: 0.5535
Entity  recall: 0.3670
Entity  F: 0.4414

#Correct Sentiment : 1036
Sentiment  precision: 0.3444
Sentiment  recall: 0.2283
Sentiment  F: 0.2746


In [946]:
a = [1,2,3]

In [947]:
a = a[::-1]

In [948]:
a

[3, 2, 1]